# 自动查询上海到郑州火车票

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import random

url='https://12306.cn/'
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('ignore-certificate-errors')

driver = webdriver.Chrome(r'C:\Users\Administrator\Documents\meijmPy\chromedriver.exe',options=options)
driver.get(url)

In [2]:
def element_input(ele_id,ele_input):
    element=driver.find_element(By.ID, ele_id)
    try:#search has no input
        element.clear()
        element.click()
        element.send_keys(ele_input)
    except:
        print(ele_id+' no input')
    element.send_keys(Keys.ENTER)
    time.sleep(1)

In [ ]:
element_input('fromStationText','上海')
element_input('toStationText','郑州')
element_input('train_date','2023-07-25')
element_input('search_one','')

In [ ]:
#drive switch 
print(driver.window_handles)
print(driver.current_window_handle)
driver.switch_to.window(driver.window_handles[1])#switch to new tab
print(driver.current_url)

In [ ]:
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
time.sleep(3)
page_text = driver.page_source
soup = BeautifulSoup(page_text, 'html.parser')

In [ ]:
find_id = soup.find(id='queryLeftTable')
columlist=['出发站','到达站','出发时间','达到时间','时长','商务座','一等座','二等座','高级软卧','软卧','动卧','硬卧','软座','硬座','无座','其他','预定']
train_df = pd.DataFrame([],columns=columlist)#to store all info
#each train info， and all train' id start with ticket in source page
tr_list=find_id.find_all(name='tr',id=re.compile("ticket_*"))
for train in tr_list:
    td_list=train.find_all('td')#each seat info
    count=0#we need to extract the train info from the first td
    seat_list=[]#to store seat info
    for treat in td_list:
        if count==0:
            train_id=treat.find(title='点击查看停靠站信息')
            print(train_id.text)
            for strong in treat.find_all('strong'):#the time and station info in strong
                print(strong.text)
                seat_list.append(strong.text)
        else:
            print(treat.text)
            seat_list.append(treat.text)
        count=count+1
    train_df1=pd.DataFrame(seat_list,index=columlist,columns=[train_id.text]).T
    train_df=pd.concat([train_df,train_df1],axis=0)
                     

In [ ]:
train_df

In [ ]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.current_url

In [19]:
def checkTrain(fromStation='上海',toStation='郑州',train_date='2023-07-25'):
    element_input('fromStationText',fromStation)
    element_input('toStationText',toStation)
    element_input('train_date',train_date)
    element_input('search_one','')
    driver.switch_to.window(driver.window_handles[1])#switch to new tab
    print(driver.current_url)
    #in case we are trecked
    time.sleep(3+random.randint(1, 9)*0.1)
    page_text = driver.page_source
    soup = BeautifulSoup(page_text, 'html.parser')
    find_id = soup.find(id='queryLeftTable')
    columlist=['出发站','到达站','出发时间','达到时间','时长','商务座','一等座','二等座','高级软卧','软卧','动卧','硬卧','软座','硬座','无座','其他','预定']
    train_df = pd.DataFrame([],columns=columlist)#to store all info
    #each train info， and all train' id start with ticket in source page
    tr_list=find_id.find_all(name='tr',id=re.compile("ticket_*"))
    for train in tr_list:
        td_list=train.find_all('td')#each seat info
        count=0#we need to extract the train info from the first td
        seat_list=[]#to store seat info
        for treat in td_list:
            if count==0:
                train_id=treat.find(title='点击查看停靠站信息')
                #print(train_id.text)
                for strong in treat.find_all('strong'):#the time and station info in strong
                    #print(strong.text)
                    seat_list.append(strong.text)
            else:
                #print(treat.text)
                seat_list.append(treat.text)
            count=count+1
        train_df1=pd.DataFrame(seat_list,index=columlist,columns=[train_id.text]).T
        train_df=pd.concat([train_df,train_df1],axis=0)
        #print(train_df)
    return train_df
                     

In [26]:
focusDate='2023-07-25'
train_df=checkTrain('上海','高安',train_date=focusDate)
train_df

search_one no input
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=%E4%B8%8A%E6%B5%B7,SHH&ts=%E9%AB%98%E5%AE%89,GCG&date=2023-07-25&flag=N,N,Y


,出发站,到达站,出发时间,达到时间,时长,商务座,一等座,二等座,高级软卧,软卧,动卧,硬卧,软座,硬座,无座,其他,预定
G1501,上海虹桥,高安,10:03,14:06,04:03,候补,候补,有,--,--,--,--,--,--,--,--,预订
G1375,上海虹桥,高安,11:11,15:41,04:30,候补,候补,有,--,--,--,--,--,--,--,--,预订
G2141,上海虹桥,高安,13:47,17:56,04:09,候补,候补,候补,--,--,--,--,--,--,--,--,预订


In [27]:
focusID="G2141"
element=driver.find_element(By.LINK_TEXT, focusID)
element.click()
time.sleep(1)
page_text = driver.page_source
soup = BeautifulSoup(page_text, 'html.parser')
find_id = soup.find(id='train_table_')
#find all staion and remove time
tr_list=find_id.find_all(text=re.compile("[\u4e00-\u9fa5]"))
pstation_list=[string for string in tr_list if '分钟' not in string ]
print(pstation_list)
driver.close()
driver.switch_to.window(driver.window_handles[0])
print(driver.current_url)

['上海虹桥', '嘉善南', '桐乡', '杭州东', '义乌', '金华', '上饶', '鹰潭北', '南昌西', '高安', '宜春', '萍乡北', '长沙南', '湘潭北', '娄底南', '怀化南', '铜仁南', '铜仁']
https://www.12306.cn/index/


In [44]:
focusdic={}
for i in range(len(pstation_list)-1):
    train_df=checkTrain(pstation_list[0],pstation_list[i+1],train_date=focusDate)
    print(pstation_list[i+1])
    print(train_df.loc[focusID]['二等座'])
    focusdic[pstation_list[i+1]]=train_df.loc[focusID]['二等座']
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    #print(driver.current_url)
print(focusdic)

search_one no input
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=%E4%B8%8A%E6%B5%B7%E8%99%B9%E6%A1%A5,AOH&ts=%E5%98%89%E5%96%84%E5%8D%97,EAH&date=2023-07-25&flag=N,N,Y
嘉善南
有
search_one no input
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=%E4%B8%8A%E6%B5%B7%E8%99%B9%E6%A1%A5,AOH&ts=%E6%A1%90%E4%B9%A1,TCH&date=2023-07-25&flag=N,N,Y
桐乡
有
search_one no input
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=%E4%B8%8A%E6%B5%B7%E8%99%B9%E6%A1%A5,AOH&ts=%E6%9D%AD%E5%B7%9E%E4%B8%9C,HGH&date=2023-07-25&flag=N,N,Y
杭州东
14
search_one no input
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=%E4%B8%8A%E6%B5%B7%E8%99%B9%E6%A1%A5,AOH&ts=%E4%B9%89%E4%B9%8C,YWH&date=2023-07-25&flag=N,N,Y
义乌
候补
search_one no input
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=%E4%B8%8A%E6%B5%B7%E8%99%B9%E6%A1%A5,AOH&ts=%E9%87%91%E5%8D%8E%E5%8D%97,RNH&date=2023-07-25&flag=N,N,Y
金华
候补
search_one no input
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs

KeyboardInterrupt: 

In [43]:
driver.close()
driver.switch_to.window(driver.window_handles[0])
print(driver.current_url)

https://www.12306.cn/index/


In [45]:
print(focusdic)

{'嘉善南': '有', '桐乡': '有', '杭州东': '14', '义乌': '候补', '金华': '候补', '上饶': '1', '鹰潭北': '候补', '南昌西': '候补', '高安': '候补', '宜春': '候补', '萍乡北': '候补', '长沙南': '5'}


In [46]:
time.sleep(3.5)

In [50]:
# 导入 random 包


# 返回一个 1 到 9 之间的数字


0.9